In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rc
import re,os
import glob, traceback
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [4]:
drug_keywords_map= {
'tobacco' : ['nicotine', 'tobacco', 'cigarette', 'cigarrette', 'cigar', 'bidis', 'snuff'],
'alcohol' : ['liquo', 'beer', 'wine'],
'cannabinoids' : ['marijuana', 'blunt', 'dope', 'ganja', 'grass', 'herb', 'joint', 'bud', 'mary jane', 'pot', 'reefer', 'green', 'trees', 'smoke', 'sinsemilla', 'skunk', 'weed','hashish', 'boom', 'gangster', 'hash', 'hash oil', 'hemp'],
'opioids' : ['heroin', 'smack', 'horse', 'brown sugar', 'dope', 
             # 'H',
             'junk', 'skag', 'skunk', 'white horse', 'China white','opium', 'laudanum', 'paregoric', 'big O', 'black stuff', 'block', 'gum', 'hop'],
'stimulants': ['cocaine', 'hydrochloride', 'blow', 'bump',
               # 'C',
                'candy', 'Charlie', 'coke', 'crack', 'flake', 'rock', 'snow', 'toot','amphetamine', 'Biphetamine', 'Dexedrine', 'bennies', 'black beauties', 'crosses', 'hearts', 'LA turnaround', 'speed', 'truck drivers', 'uppers','methamphetamine', 'Desoxyn','meth', 'ice', 'crank', 'chalk', 'crystal', 'fire', 'glass', 'go fast', 'speed'],
'club_drugs':['mdma', 'ecstasy', 'adam', 'clarity', 'eve', "lover's speed", 'peace', 'uppers','flunitrazepam', 'rohypnol', 'forget-me pill', 'mexican valium', 'r2', 'roach', 'roche', 'roffles', 'roofinol', 'rope', 'rophies','ghb', 'Gamma-hydroxybutyrate', 
              # 'G',
              'Georgia home boy', 'grievous bodily harm', 'liquid ecstasy', 'soap', 'scoop', 'goop', 'liquid X'],
'dissociative_drugs':['ketamine', 'Ketalar SV', 'cat Valium',
                      # 'K',
                      'Special K', 'vitamin K','pcp and analogs', 'phencyclidine' 'angel dust', 'boat', 'hog', 'love boat', 'peace pill','salvia divinorum', 'salvia', 'shepherdess’s herb', 'maria pastora', 'magic mint', 'sally-d','dextromethorphan', 'dxm',  'robotripping', 'robo', 'triple'],
'hallucinogens':['lsd','Lysergic acid diethylamide', 'acid', 'blotter', 'cubes', 'microdot', 'yellow sunshine', 'blue heaven', 'mescaline', 'Buttons', 'cactus', 'mesc', 'peyote', 'psilocybin', 'Magic mushrooms', 'purple passion', 'shrooms', 'little smoke'],
'other_compounds':['anabolic_steroids', 'Anadrol', 'Oxandrin', 'Durabolin', 'Depo-Testosterone', 'Equipoise', 'roids', 'juice', 'gym candy', 'pumpers','inhalants', 'Solvents', 'paint thinners', 'gasoline', 'glues', 'gasses', 'butane', 'propane', 'aerosol propellants', 'nitrous oxide',  'nitrites' ,'isoamyl', 'isobutyl', 'cyclohexyl','laughing gas', 'poppers', 'snappers', 'whippets'],
'prescription_medications':['cns_depressants', 'stimulants', 'opioid pain relievers', 'OxyContin','Oxycodone', 'Vicodin', 'Norco', 'Lortab', 'Hydrocodone', 'Acetaminophen', 'Percocet ', 'Oxycodone', 'Acetaminophen','Tramadol','Codeine','Morphine','Methadone','Demerol', 'meperidine','Acetaminophen','Tylenol', 'Excedrin', 'Vanquish','Aspirin', 'Bayer', 'Bufferin', 'Ecotrin', 'Excedrin', 'Vanquish','Diclofenac', 'Voltaren Gel','Ibuprofen', 'Advil', 'Motrin IB','Naproxen', 'Aleve']
}

columns_to_check = list(drug_keywords_map.keys())  

## 1. Read all csv files

In [9]:
all_df = []
months = [1, 2, 3, 4, 5,6, 7, 8, 9, 10, 11, 12]
#months = [1, 2]

# for month in range(10, 13):
for month in months:
    mo = f"{month:02d}"
    files = glob.glob('/data2/julina/scripts/tweets/2019/'+mo+'/pred/dm/*.csv')
    dfs = []
    for csv_file in files:
        try:
            # print(csv_file)
            a = pd.read_csv(csv_file)
            a = a.loc[:, ~a.columns.str.match('Unnamed')]
            a = a.loc[:, ~a.columns.str.match('label')]
            a = a.loc[:, ~a.columns.str.match('text_y')]
            dfs.append(a)
        except:
            print(f' failed for file : {csv_file}')
            pass
    mo_df = pd.concat(dfs, ignore_index=True)
    mo_df['date'] = pd.to_datetime('2020-' + mo)
    print(mo, '--> ', mo_df.shape)
 
    all_df.append(mo_df)
dff = pd.concat(all_df, ignore_index=True)
dff.shape

01 -->  (314802, 13)
02 -->  (262253, 13)
03 -->  (243710, 13)
04 -->  (264268, 13)
05 -->  (284107, 13)
06 -->  (287253, 13)
07 -->  (223417, 13)
08 -->  (200202, 13)
09 -->  (194789, 13)
10 -->  (177181, 13)
11 -->  (188957, 13)
12 -->  (307397, 13)


(2948336, 13)

In [10]:
dff.drop_duplicates(subset=['text', 'created_at'], inplace=True)
# dff.drop_duplicates(subset=['text', 'date'], inplace=True)

dff.shape

(2799726, 13)

In [11]:
dff['is_teenage'] = dff.apply(lambda row: 1 if row['age'] == '<=18' else 0, axis=1)

In [13]:
for keyword, words in drug_keywords_map.items():
    pattern = fr'\b(?:{"|".join(words)})\b'
    dff[keyword] = dff['text'].str.contains(pattern, case=False).astype(int)
dff['drug_type'] = dff[columns_to_check].apply(lambda row: [col for col, val in zip(columns_to_check, row) if val == 1], axis=1)
dff.drop(columns_to_check, axis=1, inplace=True)

In [14]:
dff.shape

(2799726, 15)

In [15]:
dff.to_csv('/data2/julina/scripts/tweets/cleaned_data_by_year/2019.csv')

In [24]:
# dff.sample(10)[['text', 'sentiment', 'sent_score']]
dff.head(1)

,id,created_at,text,user_id,name,screen_name,description,DrugAbuse,lang,gender,age,org,sentiment,sent_score,date,is_teenage,drug_type
0,1214331878381170694,Mon Jan 06 23:44:15 +0000 2020,nova jut give nose boop first ever boop proud ...,1034618872413020161,geralt hm fuck,lonioiHetairoi,Knell - 🇨🇦 26 - Pan - NB - they/them Nintendo ...,1,en,male,30-39,non-org,neu,0.0,2020-01-01,0,[]


## Cohort I: Teenage and female

In [103]:
words_to_remove = ['httpurl', 'hashtag', 'user']
tf_df['cleaned_text'] = tf_df['text'].apply(lambda x: ' '.join(['' if word.lower() in words_to_remove else word for word in x.split()]))

/tmp/ipykernel_2592262/4280701966.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tf_df['cleaned_text'] = tf_df['text'].apply(lambda x: ' '.join(['' if word.lower() in words_to_remove else word for word in x.split()]))


In [104]:
# tf_df.shape
vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), max_df=0.75, min_df=0.001, use_idf=True)
posts_tfidf_bow = vectorizer.fit_transform(tf_df['cleaned_text'].values.astype('U'))

In [105]:
lda_model=LatentDirichletAllocation(n_components=10,learning_method='online',random_state=42,max_iter=1) 
lda_top=lda_model.fit_transform(posts_tfidf_bow)

In [106]:
# vocab = vectorizer.get_feature_names()
vocab = vectorizer.get_feature_names_out()
for i, comp in enumerate(lda_model.components_):
     vocab_comp = zip(vocab, comp)
     sorted_words = sorted(vocab_comp, key= lambda x:x[1], reverse=True)[:20]
     print("Topic "+str(i)+": ")
     # print(sorted_words)
     for t in sorted_words:
        print(t[0], end=" ")
     print('\n')

Topic 0: 
still think covid tell alcohol hit someone first tequila enough little really bro break ever toxic long different away have 

Topic 1: 
drinking wine me drink cocaine water watch would shot 2021 cold go best eat hot mom take 20 care go get 

Topic 2: 
smoke blunt smoking feel cigarette try leave good girl amp make bottle christmas see hide he everyday put say shit 

Topic 3: 
drunk get get drunk love know stop coke lmfao night do bitch slow talk taste help lit whole walk actually body 

Topic 4: 
weed smoke please even fuck people glass start sleep smoke weed get day fun take always let bar drink drunk damn 

Topic 5: 
pain look tonight time turn today wtf every also energy hold light tree house share deserve 2020 many go almost 

Topic 6: 
it well never come one keep get could vote sex ask cigarettes head chill hope everything listen room eye high 

Topic 7: 
high need get beer right give year new thing drink wine drug get high guy nobody one work make end really 

Topic 8: 

## Question: I am trying to do Thematic analysis in twitter data. This is a cohort from teenager female users. All the tweets are identified as drug abuse from our custom DL model. Now, I have 10 topics and associated keywords from LDA, however I cannot seems to get the themes. Can you identify unique themes from following LDA result: 

To derive themes from the LDA topics you've shared, it's important to analyze the clusters of related words within each topic and consider the context in which they might be used, especially given the specific cohort (teenage female users) and the context of drug abuse. Here’s a thematic analysis based on the keywords from each topic:

### Theme 1: **Coping Mechanisms**
- **Topic 0** mentions terms like "still think covid", "alcohol", "tequila", "toxic", "break", suggesting discussions about coping with stress or mental health issues using substances.
- **Topic 5** involves "pain", "energy", "light", "house", "deserve", "2020", indicating a focus on emotional and physical states and perhaps the environment in which substance use occurs.
- **Topic 6** includes words like "keep", "hope", "chill", "eye", "high", "head", reflecting a coping theme but with a more internal, contemplative approach.

### Theme 2: **Social Contexts and Interactions**
- **Topic 1** revolves around social settings with "drinking", "wine", "drink", "shot", "mom", "eat", indicating a social or familial context.
- **Topic 3** has "drunk", "love", "night", "talk", "bitch", "lit", highlighting social interactions while under the influence, potentially at parties or gatherings.
- **Topic 8** mentions "drink", "crack", "coffee", "loudlycryingface", "miss", "happy", showing a range of emotional responses in social settings.

### Theme 3: **Substance Use and Effects**
- **Topic 2** focuses on the act and routine of substance use: "smoke", "blunt", "cigarette", "bottle", "hide", indicating habitual use and perhaps efforts to conceal this behavior.
- **Topic 4** highlights frequent substance use and its normalization: "weed", "smoke", "smoke weed", "bar", "fun", "drunk", suggesting a routine and social aspect of use.
- **Topic 7** shows a focus on addiction and regular use: "high", "get high", "beer", "drink", "drug", emphasizing dependence and routine consumption.

### Theme 4: **Reactions and Responses**
- **Topic 3** and **Topic 9** contain reactions to substance use situations, with words like "stop", "can not", "without", "know", "help", indicating resistance, help-seeking, or refusal in certain scenarios.

### Theme 5: **Lifestyle and Identity**
- **Topic 9** again, with "drunk girl", "home", "friend", "lol", "lmao", highlights a casual take on substance use, possibly as a part of identity or social image.

These themes encapsulate various aspects of substance use among teenage females as reflected in your data: coping with personal issues, socializing with peers, routine and normalized use, direct reactions to drug abuse situations, and the incorporation of substance use into their lifestyle and identity. Each theme provides a different lens through which to understand the discussions around drug abuse in the cohort, potentially guiding further qualitative analysis or intervention strategies.

## Cohort II: Teenager and Male

In [15]:
words_to_remove = ['httpurl', 'hashtag', 'user']
tm_df['cleaned_text'] = tm_df['text'].apply(lambda x: ' '.join(['' if word.lower() in words_to_remove else word for word in x.split()]))

/tmp/ipykernel_3904108/2326186899.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tm_df['cleaned_text'] = tm_df['text'].apply(lambda x: ' '.join(['' if word.lower() in words_to_remove else word for word in x.split()]))


In [16]:
# tf_df.shape
vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), max_df=0.75, min_df=0.001, use_idf=True)
posts_tfidf_bow = vectorizer.fit_transform(tm_df['cleaned_text'].values.astype('U'))

In [17]:
lda_model=LatentDirichletAllocation(n_components=10,learning_method='online',random_state=42,max_iter=1) 
lda_top=lda_model.fit_transform(posts_tfidf_bow)

In [18]:
# vocab = vectorizer.get_feature_names()
vocab = vectorizer.get_feature_names_out()
for i, comp in enumerate(lda_model.components_):
     vocab_comp = zip(vocab, comp)
     sorted_words = sorted(vocab_comp, key= lambda x:x[1], reverse=True)[:20]
     print("Topic "+str(i)+": ")
     # print(sorted_words)
     for t in sorted_words:
        print(t[0], end=" ")
     print('\n')

Topic 0: 
wine new drink start tonight leave look this eat bottle end beer heroin every water smoke you fun game new year 

Topic 1: 
it blunt coke try get right buy life even nigga good weed vote red nobody kill tire energy taste happy 

Topic 2: 
pain drinking love drug much never still street coffee drink high say tree blink 2021 make trump take cold god 

Topic 3: 
high make year not people can cigarette lol can not say well bad drive upgrade fire liquor drink find smoke weed 

Topic 4: 
drink would me cocaine please vaccine time keep back lit watch someone damn smoke break alcohol long pay listen bring 

Topic 5: 
get high lmfao really come bar way get high see turn wine drink two also best since maybe loudlycryingface head something 

Topic 6: 
crack let bro go thing stop get one smoke talk tell mean ever chill another asf anyone know stay again 

Topic 7: 
smoke alcohol weed know smoking need amp nose smoke weed day hit fuck covid house put think always 2020 next guy 

Topic 8: 

To derive themes from the LDA topics for teenage male Twitter users discussing drug abuse, we analyze the keywords within each topic while focusing on the social and behavioral nuances specific to this demographic. Here’s a thematic breakdown based on the provided keywords:

### Theme 1: **Substance Use Practices**
- **Topic 0** includes "wine", "drink", "beer", "heroin", "smoke", indicating a variety of substances used, perhaps reflecting experimentation or social drinking and smoking.
- **Topic 1** with "blunt", "coke", "weed", "kill", "energy", "taste", suggests active engagement with substances, possibly exploring effects and potency.
- **Topic 5** and **Topic 6** both highlight habitual use with "get high", "smoke", "drink", "bar", "chill", portraying both the social setting and the routine nature of substance use.

### Theme 2: **Social Interactions and Peer Influence**
- **Topic 6** emphasizes social dynamics: "let", "bro", "go", "talk", "tell", "mean", "chill", "asf", reflecting the influence of peer relationships and conversations around drug use.
- **Topic 8** also revolves around social settings with "beer", "shot", "tequila", "bitch", "everyone", indicating gatherings or parties where substance use is a shared activity.

### Theme 3: **Coping with Stress and Challenges**
- **Topic 0** and **Topic 4** show signs of using substances to cope or enhance experiences: "new year", "fun", "game", "damn", "break", "long", "pay", suggesting use in times of celebration or stress relief.
- **Topic 2** with "pain", "love", "drug", "cold", "god", indicates an emotional or spiritual seeking, perhaps using substances to deal with personal pain or existential questions.

### Theme 4: **Reactions to Societal and Personal Issues**
- **Topic 3** and **Topic 7** reflect a mix of defiance and frustration with societal norms: "can not", "bad", "fire", "fuck", "covid", "2020", showing how external events like the pandemic influence substance use behaviors and attitudes.
- **Topic 9** expresses immediate reactions and needs: "drunk", "sleep", "feel", "last", "home", emphasizing the direct impact of substances on mood and state of being.

### Theme 5: **Identity and Lifestyle**
- **Topic 7** with "smoke weed", "always", "think", "next", "guy", suggests that substance use is intertwined with personal identity and day-to-day life.
- **Topic 4** and **Topic 5** show aspects of lifestyle maintenance through substance use: "keep", "lit", "watch", "best", "loudlycryingface", portraying how substance use is integrated into daily routines and social image.

These themes provide insights into the complex ways in which teenage males discuss and engage with substance use, from social bonding and routine practices to coping mechanisms and reactions to personal and societal challenges. Each theme offers a lens to understand the subtleties of conversations around drug abuse, useful for further qualitative analysis or developing targeted interventions.

## Cohort III: Adult and Female

In [19]:
words_to_remove = ['httpurl', 'hashtag', 'user']
af_df['cleaned_text'] = af_df['text'].apply(lambda x: ' '.join(['' if word.lower() in words_to_remove else word for word in x.split()]))

/tmp/ipykernel_3904108/229630082.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  af_df['cleaned_text'] = af_df['text'].apply(lambda x: ' '.join(['' if word.lower() in words_to_remove else word for word in x.split()]))


In [21]:
# tf_df.shape
vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), max_df=0.75, min_df=0.001, use_idf=True)
posts_tfidf_bow = vectorizer.fit_transform(af_df['cleaned_text'].values.astype('U'))

In [23]:
lda_model=LatentDirichletAllocation(n_components=10,learning_method='online',random_state=42,max_iter=1) 
lda_top=lda_model.fit_transform(posts_tfidf_bow)

In [24]:
# vocab = vectorizer.get_feature_names()
vocab = vectorizer.get_feature_names_out()
for i, comp in enumerate(lda_model.components_):
     vocab_comp = zip(vocab, comp)
     sorted_words = sorted(vocab_comp, key= lambda x:x[1], reverse=True)[:20]
     print("Topic "+str(i)+": ")
     # print(sorted_words)
     for t in sorted_words:
        print(t[0], end=" ")
     print('\n')

Topic 0: 
high love get please give watch weed fuck fun end start you 2021 lit get high amp lmao smoke always kill 

Topic 1: 
smoke weed crack would vote cocaine covid life shit smoke weed tell me bad anyone think holy room eye mind listen 

Topic 2: 
blunt come need know damn back nobody never lmfao friend loudlycryingface night wine get miss drink last biden taste tire 

Topic 3: 
not can can not talk even yes eat see drink well food drunk without wait everyone anxiety hide next white literally 

Topic 4: 
take drug liquor bar bottle coke leave street find house also amp wine nigga weed drink mean morning whole use 

Topic 5: 
pain stop time do drive drink way buy trump guy cigarette drunk coffee first take hold beer this slow reason 

Topic 6: 
alcohol keep man hit much amp bring away public toxic crazy litigation full fact dad governor feeling parent drink alcohol koolaid 

Topic 7: 
drink beer year new it try tequila bitch need call shot ever two get new year work old covid maybe

## Cohort IV: Adult and Male

In [ ]:
words_to_remove = ['httpurl', 'hashtag', 'user']
am_df['cleaned_text'] = am_df['text'].apply(lambda x: ' '.join(['' if word.lower() in words_to_remove else word for word in x.split()]))

In [ ]:
# tf_df.shape
vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), max_df=0.75, min_df=0.001, use_idf=True)
posts_tfidf_bow = vectorizer.fit_transform(am_df['cleaned_text'].values.astype('U'))

In [ ]:
lda_model=LatentDirichletAllocation(n_components=10,learning_method='online',random_state=42,max_iter=1) 
lda_top=lda_model.fit_transform(posts_tfidf_bow)

In [ ]:
# vocab = vectorizer.get_feature_names()
vocab = vectorizer.get_feature_names_out()
for i, comp in enumerate(lda_model.components_):
     vocab_comp = zip(vocab, comp)
     sorted_words = sorted(vocab_comp, key= lambda x:x[1], reverse=True)[:20]
     print("Topic "+str(i)+": ")
     # print(sorted_words)
     for t in sorted_words:
        print(t[0], end=" ")
     print('\n')

## Cohort V: Org

In [27]:
words_to_remove = ['httpurl', 'hashtag', 'user']
is_org['cleaned_text'] = is_org['text'].apply(lambda x: ' '.join(['' if word.lower() in words_to_remove else word for word in x.split()]))

/tmp/ipykernel_3904108/2608925739.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  is_org['cleaned_text'] = is_org['text'].apply(lambda x: ' '.join(['' if word.lower() in words_to_remove else word for word in x.split()]))


In [28]:
# tf_df.shape
vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), max_df=0.75, min_df=0.001, use_idf=True)
posts_tfidf_bow = vectorizer.fit_transform(is_org['cleaned_text'].values.astype('U'))

In [29]:
lda_model=LatentDirichletAllocation(n_components=10,learning_method='online',random_state=42,max_iter=1) 
lda_top=lda_model.fit_transform(posts_tfidf_bow)

In [30]:
# vocab = vectorizer.get_feature_names()
vocab = vectorizer.get_feature_names_out()
for i, comp in enumerate(lda_model.components_):
     vocab_comp = zip(vocab, comp)
     sorted_words = sorted(vocab_comp, key= lambda x:x[1], reverse=True)[:20]
     print("Topic "+str(i)+": ")
     # print(sorted_words)
     for t in sorted_words:
        print(t[0], end=" ")
     print('\n')

Topic 0: 
wine take glass not can buy drink leave can not beer water tree next user liquidation come wineglass short one beermug 

Topic 1: 
smoking amp night red tonight wine last end bar guy give another pay white lose life pipe start drink world 

Topic 2: 
high thing alcohol upgrade eat low new be food use chinese sleep drink die time mean report covid alcoholic due 

Topic 3: 
drunk one say keep love joe drive get go break take sell county trump home joe biden game heroin ever house 

Topic 4: 
smoke crack go vote get know it look right try lol trump slow see blunt now money cigar walk run 

Topic 5: 
weed drink day nose first could state long hit call 2021 get drunk smoke tequila get drunk play listen voter time 

Topic 6: 
drinking 2020 year smoke christmas fire gun every good stop open eligible best well do watch without happy wine bad 

Topic 7: 
cocaine coke think would biden get cigarette much man let hunter liquor smoke fuck you via way really high kill 

Topic 8: 
drug vac